In [1]:
import boto3

In [5]:
sqs = boto3.resource('sqs', region_name='us-west-2')

In [7]:
queue = sqs.get_queue_by_name(QueueName='gis_exper_sqs_std_queue')

In [52]:
print(queue.url)
print(queue.attributes.get('DelaySeconds'))
print(queue.attributes.get('ApproximateNumberOfMessages'))
print(queue.attributes.get('ApproximateNumberOfMessagesDelayed'))
print(queue.attributes.get('ApproximateNumberOfMessagesNotVisible'))
print(queue.attributes.get('VisibilityTimeout '))

https://us-west-2.queue.amazonaws.com/241449424531/gis_exper_sqs_std_queue
0
3
0
2
None


In [9]:
response = queue.send_message(MessageBody='First GIS Message to SQS')
print(response.get('MessageId'))
print(response.get('MD5OfMessageBody'))

d7929ea2-8c49-4e24-aa8e-31e94991d3ad
2b854dea831f9dbea2c2df69ae09dad7


In [53]:
response = queue.send_messages(Entries=[
    {
        'Id': '1',
        'MessageBody': 'Second GIS Message to SQS'
    },
    {
        'Id': '2',
        'MessageBody': 'Third GIS Message to SQS',
        'MessageAttributes': {
            'Author': {
                'StringValue': 'GSA',
                'DataType': 'String'
            }
        }
    }
])

# Print out any failures
print(response.get('Failed'))

None


In [56]:
for message in queue.receive_messages(MessageAttributeNames=['Author']):
    # Get the custom author message attribute if it was set
    author_text = ''
    if message.message_attributes is not None:
        author_name = message.message_attributes.get('Author').get('StringValue')
        if author_name:
            author_text = ' ({0})'.format(author_name)

    # Print out the body and author (if set)
    print('Hello, {0}!{1}'.format(message.body, author_text))

    # Let the queue know that the message is processed
    message.delete()

Hello, Second GIS Message to SQS!


In [28]:
rs = sqs.get_available_subresources()
for r in rs:
    print(r)

Message
Queue
